# 2MASX J15220246+5010185 Astroscrappy CR Remover

## Imports

In [ ]:
# Python Imports
import os
from pathlib import Path

# Astropy Colab Imports
from astropy.io import fits
from astropy.visualization import ImageNormalize, SqrtStretch

# Other Astronomy Imports
from astroscrappy import detect_cosmics

# Plotting Imports
from matplotlib import pyplot as plt

## Notebook Setup

In [ ]:
# Check Directory
if Path.cwd().name != "Images":
    if Path.cwd().name == "Notebooks":
        os.chdir("../Images")
    else:
        RuntimeError("This notebook must be run from the Images directory.")
print(f"Current Directory: {Path.cwd()}")

In [ ]:
# Data Directory
DATA_DIR = Path('mastDownload/HST')

# FLC Glob Pattern
FLC_GLOB = list(DATA_DIR.rglob('*[!crclean]_fl?.fits'))

## Store File Name Data

In [ ]:
# Get the File Names and Sort them by filter
fileNameDict = {}
for fn in FLC_GLOB:

    # Open the file to get the filter
    with fits.open(fn) as hduList:
        hdr = hduList[0].header  # Get the Header
        if 'FILTER' in hdr:      # If the FILTER keyword exists (WFC3)
            filt = hdr['FILTER']
        elif 'CLEAR' not in hdr['FILTER1']:  # If FILTER1 is not clear (ACS)
            filt = hdr['FILTER1']
        else:                                # Else FILTER2 must be the filter (ACS)
            filt = hdr['FILTER2']

    # Store the Name using the filter as the dict key
    # Start the Empty List if Key does not exist
    if filt not in fileNameDict:
        fileNameDict[filt] = []
    fileNameDict[filt].append(fn)
fileNameDict

## Remove CRs

### Remove F814W CRs

In [ ]:
# Close Figures
plt.close('all')

# Plotting Parameters
IMG_SLICE = slice(900, 900+256), slice(700, 700+256)
norm = ImageNormalize(vmin=0, vmax=500, stretch=SqrtStretch())

# Plot a Sample Image Before CR Removal
dispFileName = fileNameDict['F814W'][0]
with fits.open(dispFileName) as hduList:
    plt.imshow(hduList['SCI', 1].data[IMG_SLICE], cmap='gray', norm=norm, origin='lower')

In [ ]:
# Input Parameters for AstroScrappy
# These parameters were found by trial and error to give good results
# See https://astroscrappy.readthedocs.io/en/latest/api.html#astroscrappy.detect_cosmics
# for more information on the parameters
ascrap_params = {
    'sigclip':      4.0,        # The sigma clipping threshold for cosmic ray detection.        Default: 4.5
    'sigfrac':      0.15,       # Fractional detection limit for neighboring pixels.            Default: 0.3
    'objlim':      10.0,        # Minimum contrast between a cosmic ray and its neighbors.      Default: 5.0
    'readnoise':    5.0,        # Read noise of the CCD in electrons.                           Default: 6.5    # 5.0 from the ACS Handbook
    'niter':        9,          # Number of iterations for the cleaning process.                Default: 4
    'sepmed':       True,       # Whether to use a separate median filter for detection.        Default: True
    'cleantype':    'medmask',  # Type of cleaning to perform ('medmask', 'medsub', or 'meanmask').  Default: 'meanmask'
    'fsmode':       'median',   # Type of filter to use ('median' or 'convolve').               Default: 'median'
    'verbose':      False,      # Whether to print verbose output.                              Default: False

    # These are only used if fsmode is 'convolve'
    'psfmodel':     'gauss',    # Type of PSF model to use ('gauss' or 'moffat').               Default: 'gauss'
    'psffwhm':      3.5,        # FWHM of the PSF model in pixels.                              Default: 2.5
    'psfsize':      11,         # Size of the PSF model array in pixels.                        Default: 7
}

# Loop Through Files
for fn in fileNameDict['F814W']:

    # Out Name
    outName = fn.with_name(fn.name.replace('flc.fits', 'crclean_flc.fits'))

    # Open Image
    with fits.open(fn) as hduList:

        # Get Cleaned Arrays
        crmsk1, crarr1 = detect_cosmics(
            hduList['SCI', 1].data, invar=hduList['ERR', 1].data**2,
            **ascrap_params
        )
        crmsk2, crarr2 = detect_cosmics(
            hduList['SCI', 2].data, invar=hduList['ERR', 2].data**2,
            **ascrap_params
        )

        # Write the Output Data
        outList = hduList.copy()  # Copy the Original File
        outList[0].header.add_history('CRs removed with DeepCR')  # Update comment about processing
        outList[0].header.add_comment('Created by Will Waldron, UAH')
        outList[0].header.add_comment('Created with pipeline at https://github.com/wwaldron/galred')
        outList['SCI', 1].data = crarr1.astype('float32')  # Change the old SCI image for CR Removed img
        outList['DQ', 1].data[crmsk1.astype(bool)] |= 2**14  # Mark DQ array as "User Mask"
        outList.insert(4, fits.ImageHDU(crmsk1.astype('uint8'), hduList['SCI', 1].header, 'MSK'))
        outList['SCI', 2].data = crarr2.astype('float32')
        outList['DQ', 2].data[crmsk1.astype(bool)] |= 2**14  # Mark DQ array as "User Mask"
        outList.insert(8, fits.ImageHDU(crmsk2.astype('uint8'), hduList['SCI', 2].header, 'MSK'))
        outList.writeto(outName, overwrite=True)

In [ ]:
# Plot a Sample Image Before CR Removal
with fits.open(dispFileName.with_name(dispFileName.name.replace('_flc', '_crclean_flc'))) as hduList:
    plt.imshow(hduList['SCI', 1].data[IMG_SLICE], cmap='gray', norm=norm, origin='lower')

### Remove F475W CRs

In [ ]:
# Close Figures
plt.close('all')

# Plotting Parameters
IMG_SLICE = slice(900, 900+256), slice(700, 700+256)
norm = ImageNormalize(vmin=0, vmax=750, stretch=SqrtStretch())

# Plot a Sample Image Before CR Removal
dispFileName = fileNameDict['F475W'][0]
with fits.open(dispFileName) as hduList:
    plt.imshow(hduList['SCI', 1].data[IMG_SLICE], cmap='gray', norm=norm, origin='lower')

In [ ]:
# Loop Through Files
for fn in fileNameDict['F475W']:

    # Out Name
    outName = fn.with_name(fn.name.replace('flc.fits', 'crclean_flc.fits'))

    # Open Image
    with fits.open(fn) as hduList:

        # Get Cleaned Arrays
        crmsk1, crarr1 = detect_cosmics(
            hduList['SCI', 1].data, invar=hduList['ERR', 1].data**2,
            **ascrap_params
        )
        crmsk2, crarr2 = detect_cosmics(
            hduList['SCI', 2].data, invar=hduList['ERR', 2].data**2,
            **ascrap_params
        )

        # Write the Output Data
        outList = hduList.copy()  # Copy the Original File
        outList[0].header.add_history('CRs removed with DeepCR')  # Update comment about processing
        outList[0].header.add_comment('Created by Will Waldron, UAH')
        outList[0].header.add_comment('Created with pipeline at https://github.com/wwaldron/galred')
        outList['SCI', 1].data = crarr1.astype('float32')  # Change the old SCI image for CR Removed img
        outList['DQ', 1].data[crmsk1.astype(bool)] |= 2**14  # Mark DQ array as "User Mask"
        outList.insert(4, fits.ImageHDU(crmsk1.astype('uint8'), hduList['SCI', 1].header, 'MSK'))
        outList['SCI', 2].data = crarr2.astype('float32')
        outList['DQ', 2].data[crmsk1.astype(bool)] |= 2**14  # Mark DQ array as "User Mask"
        outList.insert(8, fits.ImageHDU(crmsk2.astype('uint8'), hduList['SCI', 2].header, 'MSK'))
        outList.writeto(outName, overwrite=True)

In [ ]:
# Plot a Sample Image Before CR Removal
with fits.open(dispFileName.with_name(dispFileName.name.replace('_flc', '_crclean_flc'))) as hduList:
    plt.imshow(hduList['SCI', 1].data[IMG_SLICE], cmap='gray', norm=norm, origin='lower')

### Remove F275W CRs

In [ ]:
# Close Figures
plt.close('all')

# Plotting Parameters
IMG_SLICE = slice(1000, 1000+256), slice(2140, 2140+256)
norm = ImageNormalize(vmin=0, vmax=250, stretch=SqrtStretch())

# Plot a Sample Image Before CR Removal
dispFileName = fileNameDict['F275W'][0]
with fits.open(dispFileName) as hduList:
    plt.imshow(hduList['SCI', 2].data[IMG_SLICE], cmap='gray', norm=norm, origin='lower')

In [ ]:
# Input Parameters for AstroScrappy
# These parameters were found by trial and error to give good results
# See https://astroscrappy.readthedocs.io/en/latest/api.html#astroscrappy.detect_cosmics
# for more information on the parameters
ascrap_params = {
    'sigclip':      4.0,        # The sigma clipping threshold for cosmic ray detection.        Default: 4.5
    'sigfrac':      0.15,       # Fractional detection limit for neighboring pixels.            Default: 0.3
    'objlim':       5.0,        # Minimum contrast between a cosmic ray and its neighbors.      Default: 5.0
    'readnoise':    3.0,        # Read noise of the CCD in electrons.                           Default: 6.5    # 3.0 from the WFC3 Handbook
    'niter':        9,          # Number of iterations for the cleaning process.                Default: 4
    'sepmed':       True,       # Whether to use a separate median filter for detection.        Default: True
    'cleantype':    'medmask',  # Type of cleaning to perform ('medmask', 'medsub', or 'meanmask').  Default: 'meanmask'
    'fsmode':       'median',   # Type of filter to use ('median' or 'convolve').               Default: 'median'
    'verbose':      False,      # Whether to print verbose output.                              Default: False

    # These are only used if fsmode is 'convolve'
    'psfmodel':     'gauss',    # Type of PSF model to use ('gauss' or 'moffat').               Default: 'gauss'
    'psffwhm':      3.5,        # FWHM of the PSF model in pixels.                              Default: 2.5
    'psfsize':      11,         # Size of the PSF model array in pixels.                        Default: 7
}

# Loop Through Files
for fn in fileNameDict['F275W']:

    # Out Name
    outName = fn.with_name(fn.name.replace('flc.fits', 'crclean_flc.fits'))

    # Open Image
    with fits.open(fn) as hduList:

        # Get Cleaned Arrays
        crmsk1, crarr1 = detect_cosmics(
            hduList['SCI', 1].data, invar=hduList['ERR', 1].data**2,
            **ascrap_params
        )
        crmsk2, crarr2 = detect_cosmics(
            hduList['SCI', 2].data, invar=hduList['ERR', 2].data**2,
            **ascrap_params
        )

        # Write the Output Data
        outList = hduList.copy()  # Copy the Original File
        outList[0].header.add_history('CRs removed with DeepCR')  # Update comment about processing
        outList[0].header.add_comment('Created by Will Waldron, UAH')
        outList[0].header.add_comment('Created with pipeline at https://github.com/wwaldron/galred')
        outList['SCI', 1].data = crarr1.astype('float32')  # Change the old SCI image for CR Removed img
        outList['DQ', 1].data[crmsk1.astype(bool)] |= 2**14  # Mark DQ array as "User Mask"
        outList.insert(4, fits.ImageHDU(crmsk1.astype('uint8'), hduList['SCI', 1].header, 'MSK'))
        outList['SCI', 2].data = crarr2.astype('float32')
        outList['DQ', 2].data[crmsk1.astype(bool)] |= 2**14  # Mark DQ array as "User Mask"
        outList.insert(8, fits.ImageHDU(crmsk2.astype('uint8'), hduList['SCI', 2].header, 'MSK'))
        outList.writeto(outName, overwrite=True)

In [ ]:
# Plot a Sample Image Before CR Removal
with fits.open(dispFileName.with_name(dispFileName.name.replace('_flc', '_crclean_flc'))) as hduList:
    plt.imshow(hduList['SCI', 2].data[IMG_SLICE], cmap='gray', norm=norm, origin='lower')